In [1]:
import pyrealsense2 as rs2
import open3d as o3d
import numpy as np
import time
from spygrt.stream import Recording, NoMoreFrames, DualRecording
from spygrt.tracker import Tracker
from spygrt.calibration import Calibrator
import importlib
import time

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
#SET UP

rec1 = 'V:/VISION_Data//135/Recording_Session/121622061907_2023Apr12021622.bag'
rec2 = 'V:/VISION_Data/135/Recording_Session/123122061443_2023Apr12021622.bag'
cal1 = 'V:/VISION_Data/134/RS_Calibration/121622061907_2023Apr12010202.bag'
cal2 = 'V:/VISION_Data/134/RS_Calibration/123122061443_2023Apr12010202.bag'
opt1 = 'V:/VISION_Data/134/OptiTrack_Calibration_Square/121622061907_2023Apr12011406.bag'
opt2 = 'V:/VISION_Data/134/OptiTrack_Calibration_Square/123122061443_2023Apr12011406.bag'
d = DualRecording(rec1,rec2)
d.start_stream()
c = DualRecording(cal1,cal2)
c.start_stream()
calibs = (Calibrator(c.stream1), Calibrator(c.stream2))
o = DualRecording(opt1,opt2)
o.start_stream()

ERROR:root:Could not resolve the input for 'stream1'.


ValueError: Could not resolve the input for 'stream1'.

In [3]:
# CALIBRATION FOR ESTRO

for i in range (100):
    try:
        calibs[0].find_corners3d()
        calibs[1].find_corners3d()
    except NoMoreFrames:
        print("Reached the end of stream, used only the first " +str(i) + " frames for calibration.")
        break

#calibs[0].corners_scale()
#calibs[1].corners_scale()
calibs[0].align_cameras(calibs[1])
calibs[1].align_to_board(cal2 = calibs[0])
for cal in calibs:
    cal.write_cal()
    cal._pose = o3d.core.Tensor(np.identity(4), dtype=o3d.core.Dtype.Float32,device=cal._stream.pcd.device)
c.load_calibration()
for i in range (100):
    try:
        calibs[0].find_corners3d()
        calibs[1].find_corners3d()
    except NoMoreFrames:
        print("Reached the end of stream, used only the first " +str(i) + " frames for calibration.")
        break

#calibs[0].corners_scale()
#calibs[1].corners_scale()
calibs[0].align_cameras(calibs[1])
calibs[1].align_to_board(cal2 = calibs[0])
adj = o3d.core.Tensor(np.identity(4), dtype=o3d.core.Dtype.Float32, device=cal._pose.device)
adj[1][1]=-1
adj[0][0]=-1
adj[0][3] = -0.0215
adj[1][3] = -0.060
adj[2][3] = -0.120
calibs[0]._pose = adj@calibs[0].pose
calibs[1]._pose = adj@calibs[1].pose
for cal in calibs:
    cal.write_cal()

In [5]:
t = Tracker(ref_surface=d.pcd)
data = []
import time
a = time.time()
for i in range(120000):
    
    try:
        t.get_motion(d.compute_pcd(True))
    except RuntimeError:
        continue
    except NoMoreFrames:
        np.savetxt('track_135.txt',data)
        break
    nt = t.t_guess.inv()    
    data.append(np.concatenate(([d.timestamp], Tracker.compute_translation(nt), Tracker.compute_rotation(nt))))
    if i%100 == 0:
        print(i/100)
        np.savetxt('track_135.txt',data)
b = time.time()
b-a   

Timestamps:1681272982537.721 1681272982646.8347
Timestamps:1681272982537.721 1681272982646.8347
Timestamps:1681272982537.721 1681272982646.8347
Timestamps:1681272982772.5537 1681272982646.8347
0.0
Timestamps:1681272983376.3613 1681272983486.7515
Timestamps:1681272983376.3613 1681272983486.7515
Timestamps:1681272983376.3613 1681272983486.7515
Timestamps:1681272983376.3613 1681272983486.7515
Timestamps:1681272983376.3613 1681272983486.7515
Timestamps:1681272983376.3613 1681272983486.7515
Timestamps:1681272983778.9211 1681272983486.7515
Timestamps:1681272983778.9211 1681272983520.3472
Timestamps:1681272983778.9211 1681272983553.943
Timestamps:1681272983778.9211 1681272983587.539
Timestamps:1681272983778.9211 1681272983621.1348
Timestamps:1681272983778.9211 1681272983654.7307
Timestamps:1681272983778.9211 1681272983889.9011
Timestamps:1681272983778.9211 1681272983889.9011
Timestamps:1681272983778.9211 1681272983889.9011
Timestamps:1681272983778.9211 1681272983889.9011
Timestamps:1681272983

Timestamps:1681273001945.4604 1681273002065.9924
Timestamps:1681273002315.0068 1681273002435.5645
Timestamps:1681273002617.3608 1681273002502.7593
Timestamps:1681273002617.3608 1681273002502.7593
Timestamps:1681273002617.3608 1681273002502.7593
Timestamps:1681273002617.3608 1681273002872.3315
Timestamps:1681273002650.9556 1681273002872.3315
Timestamps:1681273002684.5508 1681273002872.3315
Timestamps:1681273002718.1458 1681273002872.3315
Timestamps:1681273002751.7407 1681273002872.3315
Timestamps:1681273003020.4995 1681273003141.0977
Timestamps:1681273003591.6072 1681273003745.814
Timestamps:1681273004095.5437 1681273004216.1677
Timestamps:1681273004095.5437 1681273004216.1677
Timestamps:1681273005775.6267 1681273005660.8247
Timestamps:1681273005775.6267 1681273005660.8247
Timestamps:1681273005775.6267 1681273005660.8247
Timestamps:1681273005775.6267 1681273005895.995
Timestamps:1681273006078.0059 1681273006231.968
Timestamps:1681273006111.6035 1681273006231.968
Timestamps:1681273006413

Timestamps:1681273032183.6074 1681273032572.2883
Timestamps:1681273032217.2017 1681273032572.2883
Timestamps:1681273032250.796 1681273032572.2883
Timestamps:1681273032250.796 1681273032572.2883
Timestamps:1681273032317.9841 1681273032572.2883
Timestamps:1681273032351.5781 1681273032572.2883
Timestamps:1681273032385.172 1681273032572.2883
Timestamps:1681273032418.766 1681273032572.2883
Timestamps:1681273032452.36 1681273032572.2883
Timestamps:1681273032788.299 1681273032908.2405
Timestamps:1681273033090.6946 1681273033244.2004
Timestamps:1681273033090.6946 1681273033244.2004
Timestamps:1681273033090.6946 1681273033244.2004
Timestamps:1681273034401.0002 1681273034520.8386
Timestamps:1681273034636.163 1681273034520.8386
Timestamps:1681273034669.7578 1681273034823.1924
Timestamps:1681273034669.7578 1681273034823.1924
Timestamps:1681273034669.7578 1681273034823.1924
Timestamps:1681273034669.7578 1681273034823.1924
Timestamps:1681273034669.7578 1681273034823.1924
Timestamps:1681273035072.935

Timestamps:1681273066184.4114 1681273066068.3062
Timestamps:1681273066184.4114 1681273066303.4856
Timestamps:1681273066957.1196 1681273067109.8152
Timestamps:1681273066957.1196 1681273067109.8152
Timestamps:1681273066957.1196 1681273067109.8152
Timestamps:1681273067393.8853 1681273067277.844
Timestamps:1681273067763.4563 1681273067647.513
11.0
Timestamps:1681273068670.575 1681273068789.9265
Timestamps:1681273068670.575 1681273068789.9265
Timestamps:1681273068670.575 1681273068789.9265
Timestamps:1681273068670.575 1681273068789.9265
Timestamps:1681273068972.958 1681273068789.9265
Timestamps:1681273068972.958 1681273068823.5222
Timestamps:1681273068972.958 1681273068857.118
Timestamps:1681273069174.563 1681273069058.692
Timestamps:1681273069308.967 1681273069159.4785
Timestamps:1681273069510.5745 1681273069394.6404
Timestamps:1681273069510.5745 1681273069394.6404
Timestamps:1681273069913.7886 1681273070032.932
Timestamps:1681273070182.5596 1681273070301.6921
Timestamps:1681273070518.522 

Timestamps:1681273101898.7449 1681273101781.9346
Timestamps:1681273101898.7449 1681273101781.9346
Timestamps:1681273101898.7449 1681273101781.9346
Timestamps:1681273101898.7449 1681273101781.9346
Timestamps:1681273102133.9324 1681273102252.2834
Timestamps:1681273102133.9324 1681273102252.2834
Timestamps:1681273102133.9324 1681273102252.2834
Timestamps:1681273102402.722 1681273102554.6548
Timestamps:1681273102705.1104 1681273102554.6548
Timestamps:1681273102705.1104 1681273102554.6548
Timestamps:1681273102705.1104 1681273102554.6548
Timestamps:1681273103309.8638 1681273103428.1719
Timestamps:1681273103477.8438 1681273103596.1565
Timestamps:1681273103578.6318 1681273103730.544
Timestamps:1681273103612.2278 1681273103730.544
Timestamps:1681273104149.7634 1681273104301.6897
Timestamps:1681273104418.5332 1681273104570.462
Timestamps:1681273104452.1294 1681273104570.462
Timestamps:1681273104687.3025 1681273104805.6372
Timestamps:1681273105124.0674 1681273105242.3914
Timestamps:1681273105124.

...

Timestamps:1681273621051.9358 1681273621159.0833
Timestamps:1681273621287.0547 1681273621428.0454
Timestamps:1681273621320.6594 1681273621428.0454
Timestamps:1681273621623.1047 1681273621730.385
Timestamps:1681273621623.1047 1681273621730.385
Timestamps:1681273621791.1309 1681273623040.455
Timestamps:1681273621791.1309 1681273623040.455
Timestamps:1681273621791.1309 1681273623040.455
Timestamps:1681273623571.5022 1681273623678.9097
80.0
Timestamps:1681273626023.9424 1681273627307.375
Timestamps:1681273626023.9424 1681273627307.375
Timestamps:1681273627569.2373 1681273627710.5315
Timestamps:1681273627569.2373 1681273627710.5315
Timestamps:1681273627569.2373 1681273627710.5315
Timestamps:1681273627569.2373 1681273627710.5315
Timestamps:1681273627837.992 1681273627710.5315
Timestamps:1681273628173.935 1681273628046.4924
Timestamps:1681273630189.2834 1681273630062.0022
Timestamps:1681273630189.2834 1681273630062.0022
Timestamps:1681273630189.2834 1681273630062.0022
Timestamps:1681273630189

Timestamps:1681273640235.5122 1681273640375.8433
Timestamps:1681273640470.674 1681273640577.4539
Timestamps:1681273640470.674 1681273640577.4539
Timestamps:1681273640470.674 1681273640577.4539
Timestamps:1681273640470.674 1681273640577.4539
Timestamps:1681273640705.8682 1681273640577.4539
Timestamps:1681273640705.8682 1681273640577.4539
82.0
Timestamps:1681273642520.1855 1681273642391.4175
Timestamps:1681273642520.1855 1681273642391.4175
Timestamps:1681273642520.1855 1681273642626.6094
Timestamps:1681273642822.6782 1681273642693.8186
Timestamps:1681273642822.6782 1681273642693.8186
Timestamps:1681273642822.6782 1681273642693.8186
Timestamps:1681273642822.6782 1681273642693.8186
Timestamps:1681273642822.6782 1681273642693.8186
Timestamps:1681273642822.6782 1681273642693.8186
Timestamps:1681273642822.6782 1681273642693.8186
Timestamps:1681273642822.6782 1681273642693.8186
Timestamps:1681273642822.6782 1681273642693.8186
Timestamps:1681273642822.6782 1681273642693.8186
Timestamps:16812736

83.0
[Open3D WARNING] 0 correspondence present between the pointclouds. Try increasing the max_correspondence_distance parameter.
Timestamps:1681273651390.6655 1681273652807.5698
Timestamps:1681273651390.6655 1681273652807.5698
Timestamps:1681273651390.6655 1681273652807.5698
Timestamps:1681273651390.6655 1681273652807.5698
Timestamps:1681273651390.6655 1681273652807.5698
Timestamps:1681273651390.6655 1681273652807.5698
Timestamps:1681273652969.61 1681273653076.3062
Timestamps:1681273652969.61 1681273653076.3062
Timestamps:1681273655657.2166 1681273657007.2705
Timestamps:1681273655657.2166 1681273657007.2705
Timestamps:1681273655657.2166 1681273657007.2705
Timestamps:1681273655657.2166 1681273657007.2705
Timestamps:1681273655657.2166 1681273657007.2705
Timestamps:1681273655657.2166 1681273657007.2705
Timestamps:1681273655657.2166 1681273657007.2705
Timestamps:1681273657336.7788 1681273657477.4634
Timestamps:1681273657706.2935 1681273657578.2168
Timestamps:1681273659453.3862 16812736595

Timestamps:1681273672624.4155 1681273674006.4922
Timestamps:1681273672624.4155 1681273674006.4922
Timestamps:1681273672624.4155 1681273674006.4922
Timestamps:1681273672624.4155 1681273674006.4922
Timestamps:1681273672624.4155 1681273674006.4922
Timestamps:1681273672624.4155 1681273674006.4922
Timestamps:1681273672624.4155 1681273674006.4922
Timestamps:1681273672624.4155 1681273674006.4922
Timestamps:1681273672624.4155 1681273674006.4922
Timestamps:1681273672624.4155 1681273674006.4922
Timestamps:1681273672624.4155 1681273674006.4922
Timestamps:1681273674304.0322 1681273674409.6377
Timestamps:1681273675950.9373 1681273675820.769
Timestamps:1681273676421.3213 1681273676257.4885


2650.375423669815

In [4]:
o.load_calibration(force=True)

(array([[ 0.73145878, -0.46030766,  0.50307554, -0.38821465],
        [-0.68160343, -0.51479334,  0.52000433, -0.41811669],
        [ 0.01961797, -0.72325975, -0.69029737,  0.48424962],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 array([[ 0.75355726,  0.47636428, -0.4530215 ,  0.41423345],
        [ 0.6517849 , -0.4516702 ,  0.60923761, -0.44040293],
        [ 0.08560272, -0.75436801, -0.65084642,  0.49168023],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]))

In [4]:
t = Tracker(ref_surface=d.pcd)

[Open3D INFO] Picked point #216382 (-0.078, -0.033, 0.0016) to add in queue.
[Open3D INFO] Picked point #677477 (0.081, -0.18, 0.068) to add in queue.


In [10]:
ot = Tracker(ref_surface=o.pcd)

[Open3D INFO] Picked point #848194 (-0.0089, 0.0095, -0.12) to add in queue.
[Open3D INFO] Picked point #461669 (0.0090, -0.075, -0.093) to add in queue.


In [6]:
for cal in calibs:
    cal.write_cal()

In [ ]:
[Open3D INFO] Picked point #1061769 (-0.037, 0.21, 0.023) to add in queue.
[Open3D INFO] Picked point #220126 (-0.028, 0.015, 0.027) to add in queue.
[Open3D INFO] Picked point #1023532 (-0.18, 0.013, 0.029) to add in queue.

In [7]:
data[1]

array([ 1.68127298e+12, -1.02986378e-04,  2.80050254e-05, -5.62721025e-05,
       -2.75960183e-02, -3.15382275e-02,  4.29738733e-02])